# <font color = orange><center>Youtube Comment spam</center></font>

In [113]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline , make_pipeline

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV

In [74]:
df = pd.concat( [ pd.read_csv( 'J:\\MyFiles\\Fabrice\\datascience\\data\\Youtube01.csv' ) , 
                  pd.read_csv( 'J:\\MyFiles\\Fabrice\\datascience\\data\\Youtube02.csv' ) ,
                  pd.read_csv( 'J:\\MyFiles\\Fabrice\\datascience\\data\\Youtube03.csv' ) ,
                  pd.read_csv( 'J:\\MyFiles\\Fabrice\\datascience\\data\\Youtube04.csv' ) ,
                  pd.read_csv( 'J:\\MyFiles\\Fabrice\\datascience\\data\\Youtube05.csv' ) ] )

In [37]:
df.head( 5 )

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [38]:
df[ 'CLASS' ].value_counts()

1    1005
0     951
Name: CLASS, dtype: int64

### Mélange du dataframe

In [39]:
df_shuf = df.sample( frac = 1 )

### Train / test sets

In [75]:
n = round( df_shuf.shape[0] * 0.80 )

df_train = df_shuf[ : n ]

df_test = df_shuf[ n : ] 

### ...

In [58]:
vectorizer = CountVectorizer( )

### [ 1 ] Séparation entre variables prédictives et variable à prédire

In [72]:
df_train_X = vectorizer.fit_transform( df_train[ 'CONTENT' ] )


# tranform uniquement :
df_test_X = vectorizer.transform( df_test[ 'CONTENT' ] )

In [60]:
df_train_y = df_train[ 'CLASS' ]

df_test_y = df_test[ 'CLASS' ]

### RandomForest

In [49]:
clf = RandomForestClassifier( n_estimators = 80 )

In [50]:
clf.fit( df_train_X , df_train_y )

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [51]:
clf.score( df_test_X , df_test_y )

0.9539641943734015

In [52]:
cv_score = cross_val_score( clf , df_train_X , df_train_y , cv = 10 )

print( cv_score.mean() )
print( cv_score.std() )

0.9546337527934721
0.007792958505153752


### [ 2 ] Pipeline : CountVectorizer > TFidfTransformer > RandomForest

In [104]:
df_shuf = df.sample( frac = 1 )

In [105]:
n = round( df_shuf.shape[0] * 0.80 )

In [106]:
df_X = df_shuf[ 'CONTENT' ]

df_y = df_shuf[ 'CLASS' ]

In [107]:
df_X_train = df_X[ : n ]

df_y_train = df_y[ : n ]


df_X_test = df_X[ n : ]

df_y_test = df_y[ n : ]


In [108]:
pipeline = make_pipeline( CountVectorizer() ,
                          TfidfTransformer( norm = None ) ,
                          RandomForestClassifier( n_estimators = 100 )  )

In [109]:
scores = cross_val_score( pipeline , df_X_train , df_y_train , cv = 10 )

In [110]:
print( scores.mean())
print( scores.std())

0.9552339407732875
0.015249858440119854


In [111]:
pipeline.fit( df_X_train , df_y_train )

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
  ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [112]:
pipeline.score( df_X_test , df_y_test )

0.9539641943734015

### [ 3 ] Pipeline : CountVectorizer > TFidfTransformer > RandomForest & GridSearchCV

In [114]:
df_shuf = df.sample( frac = 1 )

In [115]:
n = round( df_shuf.shape[0] * 0.80 )

In [116]:
df_X = df_shuf[ 'CONTENT' ]

df_y = df_shuf[ 'CLASS' ]

In [117]:
df_X_train = df_X[ : n ]

df_y_train = df_y[ : n ]


df_X_test = df_X[ n : ]

df_y_test = df_y[ n : ]


In [118]:
pipeline = make_pipeline( CountVectorizer() ,
                          TfidfTransformer( norm = None ) ,
                          RandomForestClassifier( n_estimators = 100 )  )

In [136]:
parameters = { 'countvectorizer__max_features' : ( None , 1000 , 2000 ) , 
               'countvectorizer__ngram_range' : ( ( 1 , 1 ) , ( 1 , 2 ) ) ,
               'countvectorizer__stop_words' : ( 'english' , None ) ,
             
               'tfidftransformer__use_idf' : ( True , False ) ,
             
               'randomforestclassifier__n_estimators' : ( 20 , 50 , 100 , 200 ) }

In [137]:
grid_search = GridSearchCV( pipeline , parameters , verbose = 1 )

In [138]:
grid_search.fit( df_X_train , df_y_train )

C:\Users\FAB\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=1)]: Done 288 out of 288 | elapsed:  2.2min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
  ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'countvectorizer__max_features': (None, 1000, 2000), 'countvectorizer__ngram_range': ((1, 1), (1, 2)), 'countvectorizer__stop_words': ('english', None), 'tfidftransformer__use_idf': (True, False), 'randomforestclassifier__n_estimators': (20, 50, 100, 200)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [139]:
grid_search.best_score_

0.9539936102236422

In [140]:
grid_search.best_estimator_.get_params()

{'memory': None,
 'steps': [('countvectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=1000, min_df=1,
           ngram_range=(1, 2), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidftransformer',
   TfidfTransformer(norm=None, smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('randomforestclassifier',
   RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
               oob_score=False, random_state=None, verbose=0,
        